In [11]:
from api4jenkins import Jenkins
import time

In [26]:
j = Jenkins('http://localhost:8080/', auth=('rtj', 'rtj@jenkins'), token=True)
j.version

'2.303.3'

In [28]:
xml = """<?xml version='1.1' encoding='UTF-8'?>
... <project>
...   <builders>
...     <hudson.tasks.Shell>
...       <command>echo $JENKINS_VERSION</command>
...     </hudson.tasks.Shell>
...   </builders>
... </project>"""

In [29]:
j.create_job('freestylejob', xml)
job = j.get_job('freestylejob')
print(job)

<FreeStyleProject: http://localhost:8080/job/freestylejob/>


In [30]:
item = j.build_job('freestylejob')

In [31]:
while not item.get_build():
     time.sleep(1)
build = item.get_build()
print(build)


<FreeStyleBuild: http://localhost:8080/job/freestylejob/1/>


In [32]:
for line in build.progressive_output():
    print(line)

Started by user Shreyansh Dubey
Running as SYSTEM
Building in workspace /var/lib/jenkins/workspace/freestylejob
[freestylejob] $ /bin/sh -xe /tmp/jenkins9788660650328168949.sh
+ echo

Finished: SUCCESS

